In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft

## Lyric generation 
Right now is done with Llama Model by using the following prompts 

In [ ]:
ollama_host = 'http://80.158.2.104:11434'
from ollama import Client
ollama_client = Client(host=ollama_host)

In [ ]:
import os
from together import Together

os.environ['TOGETHER_API_KEY'] =  "8069bcddb5b335ea3e2f23e9d58d83d5dfb270ee6ffcb8a546fbcdf8fb336dac"

together_client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))

In [ ]:
def ask_llama_3_8b_TOGETHER_API(system_intel, prompt):

    chat_completion_response = together_client.chat.completions.create(
        messages=[{"role": "system",
                   "content": system_intel},
                  {"role": "user", "content": prompt}],
        model="meta-llama/Llama-3-8b-chat-hf",
    )
    response_message = chat_completion_response.choices[0].message.content

    return response_message

In [ ]:
import os
from together import Together

os.environ['TOGETHER_API_KEY'] =  "8069bcddb5b335ea3e2f23e9d58d83d5dfb270ee6ffcb8a546fbcdf8fb336dac"

together_client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))

In [ ]:
system_intel = "You are one of the best song writers in the world.\nKnowledge cutoff: 2021-09-01\nCurrent date: 2023-03-02"
#ToDo add examples to the prompt 
prompt = """Write a set of compelling lyrics for a song and perform it with your exceptional singing skills. The lyrics should 
convey a meaningful message or tell a captivating story, and your performance should showcase your vocal abilities and emotional expression. 
Your goal is to create an original and engaging musical piece that resonates with the audience. Feel free to explore various themes and musical styles 
to demonstrate the depth of your talent as a lyricist and singer.
"""
lyrics = ask_llama_3_8b_stream_TOGETHER_API(system_intel, prompt)

In [ ]:
instruction_prompt = "Use this chorus to descriptive the possible background music that would fit it" + lyrics
instructions = ask_llama_3_8b_stream_TOGETHER_API(system_intel, lyrics)

## Music generation

In [ ]:
#Bring in the libraries 
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch

In [ ]:
#Bring in the model this will automatically download the weights 
class musicModel: 
    def __init__(self, model_type: str, audio_length) -> Audio:
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = musicgen.MusicGen.get_pretrained(model_type, device=self.device)
        self.audio_length = audio_length
        self.__set_audio_length()
    
    def __set_audio_length(self):
        self.model.set_generation_params(duration=self.audio_length)
    def generateAudio(self, instructions): 
        res = self.model.generate([
            instructions
        ], 
            progress=True)
        self.__display_audio(res)
    def __display_audio(audio): 
        display_audio(audio, 32000)
    

## Voice generation 
Reference model for voice generation: https://huggingface.co/coqui/XTTS-v2